THIS IS OUR GROUP PROJECT YAY
Code is from:
https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
#WRITE TOKEN HERE
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAABIHZgEAAAAALsxiNa1KkpHgxLPAw43%2FcbDctWQ%3DPnj1YN7wS5YPZZcZhxxv9EFqc2QrAL5iaJRm3WUlq3dUv5lOze'

In [3]:
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

#CHANGE THIS PART TO DISPLAY DIFFERENT VALUES
    query_params = {'query': keyword,
                    'max_results': max_results,
                    'expansions': 'author_id,geo.place_id', #'author_id,in_reply_to_user_id,geo.place_id', expansions controls which of the below fields you want to see
                    'tweet.fields': 'text,public_metrics,referenced_tweets,geo,created_at', #'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'name,username,created_at,description,public_metrics,verified', #'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,country,country_code,geo,name,place_type', #'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
#CHANGE
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "ucla -is:retweet lang:en" #if user, from:, #, ucla
start_time = "2022-02-01T00:00:00.000Z"
end_time = "2022-02-28T00:00:00.000Z"
max_results = 100

In [8]:
url = create_url(keyword, start_time,end_time, max_results)
print(url)
json_response = connect_to_endpoint(url[0], headers, url[1])

('https://api.twitter.com/2/tweets/search/recent', {'query': 'ucla -is:retweet lang:en', 'max_results': 100, 'expansions': 'author_id,geo.place_id', 'tweet.fields': 'text,public_metrics,referenced_tweets,geo,created_at', 'user.fields': 'name,username,created_at,description,public_metrics,verified', 'place.fields': 'full_name,country,country_code,geo,name,place_type', 'next_token': {}})
Endpoint Response Code: 200


In [9]:
json_response

{'data': [{'created_at': '2022-03-10T22:40:59.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1502051920575287296',
   'text': 'How is Hep Cronin “coaching” his son this time of year? Mick Cronin explains below:\n\nFull interview: https://t.co/9xb2HaLGSN https://t.co/l3fHXKntTX',
   'author_id': '57122589'},
  {'created_at': '2022-03-10T22:40:47.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1502051870151360518',
   'text': 'Villanova Wildcats - St. John&amp;apos;s Red Storm\nBaylor Bears - Oklahoma Sooners\nUCLA Bruins - Washington State Cougars\n\nBet slip link: https://t.co/wuKf2F51Uj https://t.co/HsjDvzG1Rv',
   'author_id': '1115823201316110336'},
  {'created_at': '2022-03-10T22:40:25.000Z',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'id': '1502051778661163009',

Change JSON to dataframe 

In [10]:
with open('imadejson.json', 'w') as outfile:
     json.dump(json_response, outfile)

In [11]:
#CHANGE THIS TO WHERE YOUR FILE IS LOCATED
file = json.load(open(r'/Users/michellepang/Desktop/projects/PIC16A-Twitter-API/imadejson.json'))

tweets = pd.DataFrame(file["data"])
users = pd.DataFrame(file["includes"]["users"])
places = pd.DataFrame(file["includes"]["places"])

In [12]:
import re
tweets["text"] = tweets["text"].apply(lambda x: re.sub( "(.+)$$(.+)" , r"\1\2",x))
print(re.sub( "(.+)$$(.+)" , r"\1\2", ""  ))
tweets

,created_at,public_metrics,id,text,author_id,referenced_tweets,geo
0,2022-03-10T22:40:59.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051920575287296,How is Hep Cronin “coaching” his son this time...,57122589,NaN,NaN
1,2022-03-10T22:40:47.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051870151360518,Villanova Wildcats - St. John&amp;apos;s Red S...,1115823201316110336,NaN,NaN
2,2022-03-10T22:40:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051778661163009,"@misterrkakes @jboe0008 @GOPLeader Let's see, ...",218027755,"[{'type': 'replied_to', 'id': '150204942130703...",NaN
3,2022-03-10T22:40:18.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051750752268304,MD E Shore +2\nVandy +5.5\nVandy over 148.5\nC...,1501786336923160582,NaN,NaN
4,2022-03-10T22:37:42.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051093152473088,@HeavenlyMariner @Basham39 @gbbcountry SCU isn...,3115848391,"[{'type': 'replied_to', 'id': '150204928305448...",NaN
...,...,...,...,...,...,...,...
95,2022-03-10T21:40:43.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502036753330311168,@mariagjMD @TrishaJamesMBA @UCLAHealth @SU2C @...,3196127461,"[{'type': 'replied_to', 'id': '150195668392853...",NaN
96,2022-03-10T21:40:38.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502036733801734145,The singles lineup 🤠\n\nLive Stats/Stream: htt...,418997165,NaN,NaN
97,2022-03-10T21:38:01.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502036074217967623,UCSC! UCLA!\nCOLA in my bank account!,1232858472963661826,NaN,NaN
98,2022-03-10T21:37:05.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502035840708587528,Proof that the brain isn’t fully developed in ...,887833427478708225,NaN,NaN


In [13]:
tweets["retweet_count"] = tweets["public_metrics"].map(lambda x:x["retweet_count"])
tweets["retweet_count"]
tweets.head()

,created_at,public_metrics,id,text,author_id,referenced_tweets,geo,retweet_count
0,2022-03-10T22:40:59.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051920575287296,How is Hep Cronin “coaching” his son this time...,57122589,NaN,NaN,0
1,2022-03-10T22:40:47.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051870151360518,Villanova Wildcats - St. John&amp;apos;s Red S...,1115823201316110336,NaN,NaN,0
2,2022-03-10T22:40:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051778661163009,"@misterrkakes @jboe0008 @GOPLeader Let's see, ...",218027755,"[{'type': 'replied_to', 'id': '150204942130703...",NaN,0
3,2022-03-10T22:40:18.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051750752268304,MD E Shore +2\nVandy +5.5\nVandy over 148.5\nC...,1501786336923160582,NaN,NaN,0
4,2022-03-10T22:37:42.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051093152473088,@HeavenlyMariner @Basham39 @gbbcountry SCU isn...,3115848391,"[{'type': 'replied_to', 'id': '150204928305448...",NaN,0


In [14]:
users

,verified,id,public_metrics,name,description,created_at,username
0,False,57122589,"{'followers_count': 400, 'following_count': 41...",Nick Koop,"PxP/Host/PA - Pac-12 Network, UCLA, UC Riversi...",2009-07-15T20:11:01.000Z,nkoop
1,False,1115823201316110336,"{'followers_count': 1811, 'following_count': 4...",Stake Sports Betting Tips,Use code TIP to get an Exclusive Welcome Offer...,2019-04-10T03:45:56.000Z,stakesportsbet
2,False,218027755,"{'followers_count': 56227, 'following_count': ...",Big Tech Tyrants Book ✝ Author: Todd Cefaratti,https://t.co/XPenc7wdUZ… The 2017 Congress and...,2010-11-21T06:23:27.000Z,ToddJCefaratti
3,False,1501786336923160582,"{'followers_count': 4, 'following_count': 5, '...",LocknessMonster,Just your average degenerate tweeting out his ...,2022-03-10T05:06:16.000Z,LocknessMonsta8
4,False,3115848391,"{'followers_count': 450, 'following_count': 39...",Gopher Crew,"Self-proclaimed CBB expert, and Gopher fan… oh...",2015-03-29T18:54:19.000Z,GopherCrew
...,...,...,...,...,...,...,...
87,False,28818172,"{'followers_count': 2625, 'following_count': 2...",Mike from the OC,Angels/Lakers/Rams/UCLA fan! Devoted caller an...,2009-04-04T16:06:24.000Z,MikefromOC
88,True,418997165,"{'followers_count': 4491, 'following_count': 5...",TCU Men's Tennis,The official Twitter account for TCU Men's Ten...,2011-11-22T20:40:15.000Z,TCUMensTennis
89,False,1232858472963661826,"{'followers_count': 93, 'following_count': 0, ...",Confused UCSC COLA Bot,I'm trying my best to support the #UCSCstrike ...,2020-02-27T02:42:29.000Z,confusedCOLAbot
90,False,887833427478708225,"{'followers_count': 47, 'following_count': 187...",illNonna,Dope.,2017-07-20T00:35:59.000Z,spicegyal76


In [15]:
places

,id,country,full_name,country_code,place_type,name,geo
0,3b77caf94bfc81fe,United States,"Los Angeles, CA",US,city,Los Angeles,"{'type': 'Feature', 'bbox': [-118.668404, 33.7..."


IDK why `places` is short. My guess is because most of the tweets aren't associated with a location hence the NaN in the geo column (like the author turned location off), and if location is on, the tweets available are only from LA.

I'm assuming users is generated based off the authors of the tweets, but I can check this because `tweets` has a column of author_id. This should match the id in `users`.

In [16]:
users["author_id"]=users["id"]

In [17]:
tweetsandusers=pd.merge(tweets,users, on="author_id")
tweetsandusers

,created_at_x,public_metrics_x,id_x,text,author_id,referenced_tweets,geo,retweet_count,verified,id_y,public_metrics_y,name,description,created_at_y,username
0,2022-03-10T22:40:59.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051920575287296,How is Hep Cronin “coaching” his son this time...,57122589,NaN,NaN,0,False,57122589,"{'followers_count': 400, 'following_count': 41...",Nick Koop,"PxP/Host/PA - Pac-12 Network, UCLA, UC Riversi...",2009-07-15T20:11:01.000Z,nkoop
1,2022-03-10T22:40:47.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051870151360518,Villanova Wildcats - St. John&amp;apos;s Red S...,1115823201316110336,NaN,NaN,0,False,1115823201316110336,"{'followers_count': 1811, 'following_count': 4...",Stake Sports Betting Tips,Use code TIP to get an Exclusive Welcome Offer...,2019-04-10T03:45:56.000Z,stakesportsbet
2,2022-03-10T22:40:25.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051778661163009,"@misterrkakes @jboe0008 @GOPLeader Let's see, ...",218027755,"[{'type': 'replied_to', 'id': '150204942130703...",NaN,0,False,218027755,"{'followers_count': 56227, 'following_count': ...",Big Tech Tyrants Book ✝ Author: Todd Cefaratti,https://t.co/XPenc7wdUZ… The 2017 Congress and...,2010-11-21T06:23:27.000Z,ToddJCefaratti
3,2022-03-10T22:40:18.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051750752268304,MD E Shore +2\nVandy +5.5\nVandy over 148.5\nC...,1501786336923160582,NaN,NaN,0,False,1501786336923160582,"{'followers_count': 4, 'following_count': 5, '...",LocknessMonster,Just your average degenerate tweeting out his ...,2022-03-10T05:06:16.000Z,LocknessMonsta8
4,2022-03-10T22:37:42.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502051093152473088,@HeavenlyMariner @Basham39 @gbbcountry SCU isn...,3115848391,"[{'type': 'replied_to', 'id': '150204928305448...",NaN,0,False,3115848391,"{'followers_count': 450, 'following_count': 39...",Gopher Crew,"Self-proclaimed CBB expert, and Gopher fan… oh...",2015-03-29T18:54:19.000Z,GopherCrew
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-03-10T21:41:39.000Z,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",1502036990228795407,@GuruLakers The same group who barely recogniz...,28818172,"[{'type': 'replied_to', 'id': '150202813029846...",NaN,1,False,28818172,"{'followers_count': 2625, 'following_count': 2...",Mike from the OC,Angels/Lakers/Rams/UCLA fan! Devoted caller an...,2009-04-04T16:06:24.000Z,MikefromOC
96,2022-03-10T21:40:38.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502036733801734145,The singles lineup 🤠\n\nLive Stats/Stream: htt...,418997165,NaN,NaN,0,True,418997165,"{'followers_count': 4491, 'following_count': 5...",TCU Men's Tennis,The official Twitter account for TCU Men's Ten...,2011-11-22T20:40:15.000Z,TCUMensTennis
97,2022-03-10T21:38:01.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502036074217967623,UCSC! UCLA!\nCOLA in my bank account!,1232858472963661826,NaN,NaN,0,False,1232858472963661826,"{'followers_count': 93, 'following_count': 0, ...",Confused UCSC COLA Bot,I'm trying my best to support the #UCSCstrike ...,2020-02-27T02:42:29.000Z,confusedCOLAbot
98,2022-03-10T21:37:05.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1502035840708587528,Proof that the brain isn’t fully developed in ...,887833427478708225,NaN,NaN,0,False,887833427478708225,"{'followers_count': 47, 'following_count': 187...",illNonna,Dope.,2017-07-20T00:35:59.000Z,spicegyal76


Since this df is 100 rows, the same as the tweets, I think this means every tweet has a user that was in `users`. Next step is maybe filtering the tweets. Like looking at tweets with at least 10 likes, or looking at not retweeted tweets.

work on clusters: grouping topics together


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words="english")
counts = vec.fit_transform(tweetsandusers['text'])
counts = counts.toarray()
counts

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
vec.get_feature_names()

['00',
 '00pm',
 '10',
 '100',
 '11',
 '110',
 '112',
 '12',
 '120',
 '124',
 '13',
 '130',
 '132',
 '14',
 '141',
 '148',
 '15',
 '16',
 '1929',
 '1966',
 '1992',
 '1ck5vpjiyj',
 '1h',
 '1u',
 '2017',
 '2018',
 '2022',
 '2023',
 '227',
 '24',
 '2ausapxrro',
 '2obeto',
 '30',
 '311',
 '3m8jxbsqgm',
 '3u',
 '484',
 '4842',
 '4cqsontoi8',
 '4rnatnd4wc',
 '4rqrqjtyyf',
 '50',
 '5lelvocxpt',
 '5thdowncfb',
 '61',
 '62',
 '63',
 '66',
 '6qlacykg2h',
 '6uelc0mvi2',
 '703',
 '75',
 '76ers',
 '787',
 '7bs9fbstj0',
 '7owqof11bd',
 '855',
 '85nbgxjn0j',
 '8neazwmxli',
 '8nnqnm4duh',
 '9xb2halgsn',
 'abc',
 'able',
 'abundantly',
 'academics_real',
 'accepted',
 'accidentally',
 'account',
 'act',
 'action',
 'added',
 'address',
 'addressing',
 'admitted',
 'adorable',
 'ah',
 'ahead',
 'ahktpqt9tv',
 'air',
 'aiu',
 'alabama',
 'alumniaction',
 'amazing',
 'amp',
 'ana',
 'ancient',
 'angeles',
 'antidote',
 'aou',
 'apos',
 'arboledaval',
 'argue',
 'argument',
 'arizona',
 'arkansas',
 'array

In [20]:
count_tweetsandusers = pd.DataFrame(counts, columns=vec.get_feature_names())
count_tweetsandusers

,00,00pm,10,100,11,110,112,12,120,124,...,xtajrx4c2e,xula24,xzbcni1ihh,yara,year,years,yesqueenjess,young,youtube,zachalucas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [21]:
big_count_tweetsandusers = pd.concat((tweetsandusers["text"],count_tweetsandusers), axis= 1)
big_count_tweetsandusers

,text,00,00pm,10,100,11,110,112,12,120,...,xtajrx4c2e,xula24,xzbcni1ihh,yara,year,years,yesqueenjess,young,youtube,zachalucas
0,How is Hep Cronin “coaching” his son this time...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Villanova Wildcats - St. John&amp;apos;s Red S...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"@misterrkakes @jboe0008 @GOPLeader Let's see, ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MD E Shore +2\nVandy +5.5\nVandy over 148.5\nC...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,@HeavenlyMariner @Basham39 @gbbcountry SCU isn...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,@GuruLakers The same group who barely recogniz...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
96,The singles lineup 🤠\n\nLive Stats/Stream: htt...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,UCSC! UCLA!\nCOLA in my bank account!,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Proof that the brain isn’t fully developed in ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
X = big_count_tweetsandusers.drop(columns = "text")
from sklearn.decomposition import NMF
model = NMF(n_components = 10, init="random", random_state=0)
model.fit(X)

NMF(init='random', n_components=10, random_state=0)

In [23]:
import numpy as np
def top_words(X, model, component, num_words):
    orders = np.argsort(model.components_, axis = 1)
    important_words = np.array(X.columns)[orders]
    return important_words[component][-num_words:]

In [24]:
for i in range (10):
    print(top_words(X, model, i, 8))

['227' 'k8jh27syht' 'ucla' 'bruins' 'mix' 'spicy' 'chili' 'https']
['bel' 'die' 'needed' 'sepulveda' 'la' 'stop' 'https' 'ceo']
['notre' 'dame' 'colorado' 'let' 'day' '1h' '110' '1u']
['estate' 'español' 'hispanics' 'people' 'ucla' 'let' 'visit' 'https']
['kerr' 'vs' 'tourney' 'season' 'ucla' 'tournament' 'pac' '12']
['higher' 'slotted' 'seed' 'make' 'run' 'play' '10' 'like']
['1h' 'ucla' 'usc' 'clara' 'santa' 'road' 'win' 'vs']
['https' 'graduate' 'black' 'ml' 'just' 'students' 'campus' 'ucla']
['nyu' 'https' 'ucla' 'study' 'good' 'case' 'history' 'essay']
['spoke' 'love' 'ucla' 'today' 'wonderful' 'center' 'worker' 'amp']


In [25]:
for i in range (10):
    print(top_words(X, model, i, 4))

['mix' 'spicy' 'chili' 'https']
['la' 'stop' 'https' 'ceo']
['day' '1h' '110' '1u']
['ucla' 'let' 'visit' 'https']
['ucla' 'tournament' 'pac' '12']
['run' 'play' '10' 'like']
['santa' 'road' 'win' 'vs']
['just' 'students' 'campus' 'ucla']
['good' 'case' 'history' 'essay']
['wonderful' 'center' 'worker' 'amp']


In [26]:
big_count_tweetsandusers.columns

Index(['text', '00', '00pm', '10', '100', '11', '110', '112', '12', '120',
       ...
       'xtajrx4c2e', 'xula24', 'xzbcni1ihh', 'yara', 'year', 'years',
       'yesqueenjess', 'young', 'youtube', 'zachalucas'],
      dtype='object', length=1097)

In [27]:
from matplotlib import pyplot as plt
document_weights = model.transform(X)
fig, ax = plt.subplots(1)
for i in range(10):
    ax.plot(tweetsandusers['text'], document_weights[:,i], label = top_words(X, model, i, 8))
ax.legend(bbox_to_anchor=(1.05, 0.65), loc="upper left")
plt.show()

/Users/michellepang/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 128680 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/Users/michellepang/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 128293 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/Users/michellepang/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 129315 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/Users/michellepang/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 128187 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/Users/michellepang/opt/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_agg.py:240: RuntimeWarning: Glyph 128104 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/Users/michellepang/opt/a

ValueError: 
@UConnnation12 @bnpparibaopen @JonRothstein Nah I’ll stick with what I said in regards to Kerr’s impact on the game. Whenever Arizona has lost this season, go straight to the stat sheet &amp; you see Kerr shot 2 for 15. Watch the games &amp; he’s taking bad shots. UCLA is still the Pac-12 team to put ur $$ on for the tournament
                                                                                                                                                                                                                                                                                                                 ^
Expected end of text, found '$'  (at char 305), (line:1, col:306)

<Figure size 432x288 with 1 Axes>